### ML
Суть решения в том, чтобы собрать статистики по пользователю и по предмету и к каждой комбинации этих статистик предсказывать 0 или 1  
Использовался CatBoost - градиентный бустинг над решающими деревьями  
+Также добавлялись сюда рекомендации от TIFU KNN, это дало хороший прирост +0.08 к метрике, в итоге результат около 0.40, как в случае и с TIFU KNN

! Можно запускать в режиме "Run all"

In [1]:
%%time
%pylab inline
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()

Populating the interactive namespace from numpy and matplotlib
Wall time: 3.06 s


c:\envs\kate\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Unpack the data

In [2]:
df = pd.read_csv("data/main.csv")

df.rename(columns={"order_completed_at":"time"}, inplace=True) # rename "order_completed_at" column to "time"
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S") # "time" column to datetime type

Тут те же правки по функциям, как и в ноутбуке Typical baselines.

In [3]:
def duplicates_to_count(t):
    return t.groupby(['user_id', 'time'])['cart'].value_counts() \
                                                  .to_frame() \
                                                  .rename(columns={"cart":"count"}) \
                                                  .reset_index()

def count_to_duplicates(t):
    g = t.copy()
    g["to_explode"] = g["count"].apply(lambda x: [i for i in range(x)])
    g = g.explode("to_explode") \
         .drop(columns=["count", "to_explode"])
    return g

def make_train_targets(t, level=1):
    user_last_time = t.groupby(["user_id"])["time"].max().to_frame().reset_index()
    user_last_time["last_buy"] = 1
    
    train = pd.merge(t, user_last_time, on=["time", "user_id"], how="left")
    train = train[train["last_buy"] != 1]
    train.drop(columns=["last_buy"], inplace=True)
    
    if level >= 2:
        return make_train_targets(train, level-1)
    
    user_last_time.drop(columns=["last_buy"], inplace=True)
    
    user_last_carts = pd.merge(user_last_time, t, on=["user_id", "time"], how="left")
    
    skeleton = make_skeleton(t)
    targets = pd.merge(skeleton, user_last_carts.drop(columns=["time"]), on=["user_id","cart"], how="left")
    targets.fillna(0, inplace=True)
    targets["count"] = targets["count"].progress_apply(lambda x: x if x <= 1 else 1).astype(int)
    targets.rename(columns={"count":"target"}, inplace=True)
    return train, targets

def make_skeleton(t):
    return t.groupby("user_id")["cart"].unique().to_frame().reset_index().explode("cart")


##### Что можно сделать

Поработать над фичами, которые отражают взаимодействие конкретного пользователя и конкретного товара.

In [12]:
# Топ категорий пользователя
def user_top_k_carts(t, k):
    g = count_to_duplicates(t).groupby("user_id")["cart"].value_counts().to_frame().rename(columns={"cart":"count"})\
        .groupby("user_id")["count"].head(k).to_frame().reset_index().drop(columns=["count"])\
        .groupby("user_id")["cart"].agg([lambda x: x.tolist()]).rename(columns={"<lambda>":"user_top_"+str(k)+"_carts"}).reset_index()
    for i in tqdm(range(k)):
        g["user_top_"+str(i+1)+"_cart"] = g["user_top_"+str(k)+"_carts"].apply(lambda x: x[i] if len(x) > i else -1)
    return g.drop(columns=["user_top_"+str(k)+"_carts"]) 

# Кол-во заказов пользователя
def user_orders_count(t):
    return t.groupby(["user_id"])["time"].nunique().to_frame().reset_index().rename(columns={"time":"orders_count"})

# Кол-во вещей пользователя по всем заказам
def user_items_count(t):
    return t.groupby("user_id")["count"].sum().to_frame().reset_index().rename(columns={"count":"items_count"})

# Таблица как в TIFU KNN, сильно грузит память (около 30GB RAM), но прироста большого не дает
def user_pivot_table(t):
    g = pd.pivot_table(t, columns="cart", index="user_id", values="count", aggfunc=np.sum, fill_value=0)
    for cart in list(set(df["cart"].unique()).difference(set(t["cart"].unique()))):
        g[cart] = 0
    return g

# Количество уникальных категорий, в которых покупал пользователь
def user_carts_count(t):
    return t.groupby("user_id")["cart"].nunique().to_frame().reset_index().rename(columns={"cart":"carts_count"})

# Кол-во заказов каждой категории
def cart_orders_count(t):
    return t.groupby("cart")["count"].agg(["sum"]).reset_index().rename(columns={"sum":"cart_orders_count"})

# Кол-во пользователей, которые заказали товар из категории
def cart_unique_users_count(t):
    return t.groupby("cart")["user_id"].nunique().to_frame().reset_index().rename(columns={"user_id":"cart_unique_users_count"})

# Среднее, мин., макс. и медиана количества товаров каждой категории в корзине (бесполезная)
def cart_mean_count(t):
    return t.groupby("cart")["count"].agg([("cart_mean_count","mean"),
                                           ("cart_min_count", "min"),
                                           ("cart_max_count", "max"),
                                           ("cart_median_count", "median")]).reset_index()

# Среднее, мин., макс. и медиана размера корзины пользователя за все заказы
def user_mean_cart_in_order_count(t):
    return t.groupby(["user_id","time"])["count"].sum().to_frame() \
                                                 .groupby(["user_id"]) \
                                                 .agg(user_mean_cart_in_order_count = ('count', 'mean'),
                                                      user_min_cart_in_order_count = ('count', 'min'),
                                                      user_max_cart_in_order_count = ('count', 'max'),
                                                      user_median_cart_in_order_count = ('count', 'median')).reset_index()
    
def make(df, skeleton):
    main = skeleton.copy()
    
    main = pd.merge(main, user_orders_count(df), on="user_id", how="left")
    main.fillna(0, inplace=True)
    main["orders_count"] = main["orders_count"].astype(int)
    
    main = pd.merge(main, user_items_count(df), on="user_id", how="left")
    main.fillna(0, inplace=True)
    main["items_count"] = main["items_count"].astype(int)
    
    main = pd.merge(main, user_carts_count(df), on="user_id", how="left")
    main.fillna(0, inplace=True)
    main["carts_count"] = main["carts_count"].astype(int)
    
    main = pd.merge(main, cart_orders_count(df), on="cart", how="left")
    main.fillna(0, inplace=True)
    main["cart_orders_count"] = main["cart_orders_count"].astype(int)
    
    main = pd.merge(main, cart_unique_users_count(df), on="cart", how="left")
    main.fillna(0, inplace=True)
    main["cart_unique_users_count"] = main["cart_unique_users_count"].astype(int)
    
    main = pd.merge(main, user_mean_cart_in_order_count(df), on="user_id", how="left")
    main.fillna(0, inplace=True)
    
    main = pd.merge(main, cart_mean_count(df), on="cart", how="left")
    main.fillna(0, inplace=True)
    
    top_k = 5
    main = pd.merge(main, user_top_k_carts(df, top_k), on="user_id", how="left")
    main.fillna(-1, inplace=True)
    for i in range(top_k):
        main["user_top_"+str(i+1)+"_cart"] = main["user_top_"+str(i+1)+"_cart"].astype(int)
    

    return main

In [13]:
df = duplicates_to_count(df)

In [14]:
train_1, targets_1 = make_train_targets(df, level=1)

main_1 = make(train_1, targets_1)

main_1.head()

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 156.65it/s]


,user_id,cart,target,orders_count,items_count,carts_count,cart_orders_count,cart_unique_users_count,user_mean_cart_in_order_count,user_min_cart_in_order_count,...,user_median_cart_in_order_count,cart_mean_count,cart_min_count,cart_max_count,cart_median_count,user_top_1_cart,user_top_2_cart,user_top_3_cart,user_top_4_cart,user_top_5_cart
0,0,14,0,2,33,27,85164,15273,16.5,8,...,16.5,1.0,1.0,1.0,1.0,14,57,82,379,405
1,0,20,0,2,33,27,13664,5906,16.5,8,...,16.5,1.0,1.0,1.0,1.0,14,57,82,379,405
2,0,57,1,2,33,27,98788,16336,16.5,8,...,16.5,1.0,1.0,1.0,1.0,14,57,82,379,405
3,0,82,0,2,33,27,24980,7493,16.5,8,...,16.5,1.0,1.0,1.0,1.0,14,57,82,379,405
4,0,379,0,2,33,27,19764,8470,16.5,8,...,16.5,1.0,1.0,1.0,1.0,14,57,82,379,405


##### Что нужно сделать

Перезаписать таблички с рекомендациями tifu knn

In [15]:
recs_1 = pd.read_csv("data/train_lvl_1_recs.csv")

In [16]:
recs_1.head()

,user_id,items,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,"[57, 14, 82, 84, 430, 22, 409, 61, 379, 441, 3...",57,14,82,84,430,22,409,61,379,441,382,41,5,383,398
1,1,"[55, 798, 812, 169, 14, 88, 170, 171, 198, 23,...",55,798,812,169,14,88,170,171,198,23,404,19,406,57,61
2,2,"[57, 61, 23, 14, 409, 84, 82, 425, 398, 22, 43...",57,61,23,14,409,84,82,425,398,22,430,403,384,382,16
3,3,"[61, 57, 84, 398, 16, 430, 14, 22, 383, 382, 3...",61,57,84,398,16,430,14,22,383,382,399,41,43,23,402
4,4,"[57, 61, 398, 84, 54, 22, 712, 17, 100, 388, 4...",57,61,398,84,54,22,712,17,100,388,420,383,425,16,14


In [17]:
main_1 = pd.merge(main_1, recs_1, on="user_id", how="left")

In [23]:
main_array = main_1.to_numpy()
arr_has = []
arr_pos = []
index_of_items = main_1.columns.tolist().index("items")
for row in tqdm(main_array):
    v = row[index_of_items].replace(']','').replace('[','')
    v = v.split(", ")
    for i in range(len(v)):
        v[i] = int(v[i])
    if row[1] in v:
        arr_has.append(1)
        arr_pos.append(v.index(row[1]))
    else:
        arr_has.append(0)
        arr_pos.append(-1)
        
main_1["has_cart_in_recs"] = arr_has
main_1["pos_cart_in_recs"] = arr_pos

100%|████████████████████████████████████████████████████████████████████| 1117600/1117600 [00:06<00:00, 172449.69it/s]


In [25]:
main_1.drop(columns=["items"], inplace=True)

Далее можно расширить обучающую выборку, добавив к ней датасет в таком же формате, но собраный при level=2, например

##### (Я бы сказала, что в нашем случае датасет, собранный при level=2, нужно использовать как обучающую выборку; датасет при level=1 - как тест

Нужно дополнительно подумать, как кодировать категориальные фичи, чтобы это реализовать, т.к. catboost не будет давать предикт, если появятся новые значения у категориальной фичи.

In [26]:
train_2, targets_2 = make_train_targets(df, level=2)

main_2 = make(train_2, targets_2)

recs_2 = pd.read_csv("data/train_lvl_2_recs.csv")

main_2 = pd.merge(main_2, recs_2, on="user_id", how="left")

main_array = main_2.to_numpy()
arr_has = []
arr_pos = []
index_of_items = main_2.columns.tolist().index("items")
for row in tqdm(main_array):
    v = row[index_of_items].replace(']','').replace('[','')
    v = v.split(", ")
    for i in range(len(v)):
        v[i] = int(v[i])
    if row[1] in v:
        arr_has.append(1)
        arr_pos.append(v.index(row[1]))
    else:
        arr_has.append(0)
        arr_pos.append(-1)
        
main_2["has_cart_in_recs"] = arr_has
main_2["pos_cart_in_recs"] = arr_pos

main_2.drop(columns=["items"], inplace=True)

100%|████████████████████████████████████████████████████████████████████| 1031269/1031269 [00:07<00:00, 132648.17it/s]


In [51]:
main_final = main_1.copy()

In [52]:
main_final.shape

(1117600, 38)

Cart - тоже категориальная фича.

In [53]:
top_k = 5
cat_features = ['cart'] + ["has_cart_in_recs"] + ["user_top_"+str(i+1)+"_cart" for i in range(top_k)] + [str(i) for i in range(15)]
cat_features

['cart',
 'has_cart_in_recs',
 'user_top_1_cart',
 'user_top_2_cart',
 'user_top_3_cart',
 'user_top_4_cart',
 'user_top_5_cart',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14']

In [54]:
x_train, x_validation, y_train, y_validation = train_test_split(main_final.drop(columns=["user_id", "target"]), 
                                                                main_final["target"], 
                                                                stratify=main_final["target"],
                                                                test_size=0.33, 
#                                                                 random_state=42
                                                               )

In [55]:
model = CatBoostClassifier(iterations=300,
#                             depth = 6,
                            learning_rate = 0.35,
#                             l2_leaf_reg = 4,
                            eval_metric="F1",
                            loss_function = "Logloss",
                            task_type="GPU",
                            # fold_permutation_block = 2,
                            # fold_len_multiplier = 1.5,
                            # leaf_estimation_iterations = 10,
                            # max_ctr_complexity = 1,
                            random_seed= 127,
                            cat_features = cat_features
                           )

In [56]:
model.fit(x_train, 
          y_train, 
          eval_set=(x_validation, y_validation), 
          use_best_model=True, 
          early_stopping_rounds=50,  
#           plot=True, 
          verbose=10
          )

0:	learn: 0.0811401	test: 0.0677372	best: 0.0677372 (0)	total: 1.52s	remaining: 7m 34s
10:	learn: 0.2298165	test: 0.2529617	best: 0.2529617 (10)	total: 17.3s	remaining: 7m 33s
20:	learn: 0.2933796	test: 0.3127299	best: 0.3167296 (18)	total: 34.4s	remaining: 7m 37s
30:	learn: 0.3218773	test: 0.3372736	best: 0.3382047 (29)	total: 51.2s	remaining: 7m 24s
40:	learn: 0.3413587	test: 0.3525930	best: 0.3531521 (36)	total: 1m 8s	remaining: 7m 9s
50:	learn: 0.3496677	test: 0.3609564	best: 0.3609564 (50)	total: 1m 25s	remaining: 6m 58s
60:	learn: 0.3576393	test: 0.3665030	best: 0.3670108 (57)	total: 1m 42s	remaining: 6m 41s
70:	learn: 0.3624867	test: 0.3695951	best: 0.3698960 (66)	total: 1m 59s	remaining: 6m 25s
80:	learn: 0.3641519	test: 0.3693316	best: 0.3702325 (78)	total: 2m 15s	remaining: 6m 7s
90:	learn: 0.3675144	test: 0.3704833	best: 0.3704833 (90)	total: 2m 32s	remaining: 5m 51s
100:	learn: 0.3692168	test: 0.3712727	best: 0.3719728 (99)	total: 2m 49s	remaining: 5m 33s
110:	learn: 0.3707

In [17]:
# Score: 0.39940 (level=1), 0.48547 (level=2), 0.59612 (level=3) - mean=0.49366
# 0.44531 (level=1;2)

In [57]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,cart,25.060299
1,orders_count,24.008379
2,carts_count,9.480433
3,pos_cart_in_recs,8.778133
4,has_cart_in_recs,6.310312
5,user_top_5_cart,4.005802
6,items_count,3.846434
7,13,2.258012
8,user_median_cart_in_order_count,2.212467
9,14,2.167287


In [58]:
useless_features = model.get_feature_importance(prettified=True).query("Importances==0")["Feature Id"].tolist()

In [59]:
useless_features

['cart_orders_count',
 'cart_unique_users_count',
 'user_max_cart_in_order_count',
 'cart_min_count',
 'cart_median_count']

##### Что еще можно попробовать

Можно попробовать сделать кластеризацию покупателей и делать модельки для каждого кластера.

Можно кластеризовать товары, используя в качестве фичей эмбеддинги, построенные по корзинам покупателей. Это позволит делать рекомендации товаров, которые ранее не покупались данным клиентом.